In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import os
import re 



path = os.getcwd()+'\\data\\redit_comments\\'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
data_files=[]
for e in onlyfiles:
    if '.csv' in e:
        data_files.append(e)

df=pd.DataFrame()

print('Loading data')

for e in tqdm(data_files):
    df1=pd.read_csv(path+e,low_memory=False)

    df=pd.concat([df,df1])

print(F"Unique rows of df: {len(df.body.unique())}")
print(F"Number of  rows : {len(df)}")

#Droping duplicatees values in the desired column(s)
df=df.drop_duplicates(subset = ["body"])
print(F"Number of  rows  after droping duplicates: {len(df)}")


remove_list=['[deleted]','[removed]']
#drop all rows containing the elements of the list
df = df[~df['body'].isin(remove_list)]

print(F"Number of  rows  (final): {len(df)}")

#Making a copy of the original text before cleaning the data

#shuffle the rows getting only a fraction of the data
df=df.sample(frac=1)


print(F"Number of  rows  after sample: {len(df)}")

df=df.reset_index(drop=True)

100%|██████████| 7/7 [00:17<00:00,  2.57s/it]
Unique rows of df: 739623
Number of  rows : 845356
Number of  rows  after droping duplicates: 739623
Number of  rows  (final): 739621
Number of  rows  after sample: 739621


In [2]:
df.body=df.body.str.replace("\n"," ").replace("\r", " ").replace('"',"'")

df=df[['body','parent_id','author_fullname','id','permalink','link_id','author','is_submitter','send_replies','score','subreddit']]


In [3]:
df.parent_id=df.parent_id.str.replace(r".*_",'')
df.link_id=df.link_id.str.replace(r".*_",'')

<ipython-input-3-12284bca5a7e>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.parent_id=df.parent_id.str.replace(r".*_",'')
<ipython-input-3-12284bca5a7e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.link_id=df.link_id.str.replace(r".*_",'')


# Cleaning the data

In [4]:
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt

In [5]:
df.body=df.body.astype(str).apply(lambda x: clean_text(x))


In [6]:
df.body


0         any language that runs in a virtual machine or...
1         its nice when you get really comfortable with ...
2         thank you very much im sure this will lead me ...
3         such a star would have a negative temperature ...
4         vectors are 1dimensional so two vectors can sp...
                                ...                        
739616    i dont think taking field theory will clear up...
739617    i have seen this before and i was still please...
739618    there is always a backup plan i graduated from...
739619    in finance especially commodities we deal with...
739620    even if you can separate a class system from a...
Name: body, Length: 739621, dtype: object

In [7]:
comment_list=[]
response_list=[]
comment_score=[]
response_score=[]
ids=df.id

get_matches=True

if get_matches:
    for i in tqdm(range(len(df))):
        #comment=df2.loc[df2.id==ids[i]].body.values[0]
        comment=df.iloc[i].body


        #comment with the highest score 
        try:
            argmax_score=df.loc[df.parent_id==ids[i]].score.idxmax()
            response=df.iloc[argmax_score].body
        #If there is no  response 
        except:
            response=None
        #print(response)

        if response !=None:
            comment_score.append(int(df.iloc[i].score))
            comment_list.append(comment)

            response_list.append(response)
            response_score.append(int(df.iloc[argmax_score].score))

data=pd.DataFrame([comment_list,comment_score,response_list,response_score]).T

data.columns=['Comment','comment_score','response','response_score']
#data.columns=['Comment','response',]

data

  7%|▋         | 54794/739621 [52:21<12:23:39, 15.35it/s]

In [44]:
data.iloc[308].values

array(['sir this is wendys', 20,
       'bet you can find some of this at a wendys if you look close enough ',
       18], dtype=object)

In [3]:
loc=os.getcwd()+'\\data\\paired_comments\\'

save_to_csv=False
if save_to_csv:
    data.to_csv(loc)


In [13]:
df.body.values

array(['i hope it is something that can not be predicted by any theorymodel we have that should give us a good boost',
       'for 1 it is simply definitions we have defined the speed of light using a different definition of meter and using this maximum speed we decided to define the meter as being the distance that light travels in time t  1c for obvious reasons which is more consistent over time than using a stick',
       'the panspermia hypothesis is also not saying that the elements needed for life came from space  its about complex organic molecules like sugars that make up the basis of the simplest lifeforms which have been shown to form in space and have been found in meteorites  its just saying the molecules didnt form on earth and life got kickstarted by molecules from somewhere else',
       ...,
       'come on man its feynman how can anyone not have time for feynman go on read them you know you want to ',
       'doesnt the ground support your weight when you close your ey